Генерация FP 4 - таблиц но без ieee-754

In [1]:
BIAS = 1
EMIN = -1   
EMAX =  2   


def fp4_to_float(x: int) -> float:
    sign = (x >> 3) & 0b1
    exp_bits = (x >> 1) & 0b11
    frac = x & 0b1

    if exp_bits == 0 and frac == 0:
        return 0.0 * (-1) ** sign

    E = exp_bits - BIAS
    mant = 1.0 + frac * 0.5
    return ((-1) ** sign) * mant * (2 ** E)

In [2]:
for x in range(1 << 4):
    print(bin(x), fp4_to_float(x))

0b0 0.0
0b1 0.75
0b10 1.0
0b11 1.5
0b100 2.0
0b101 3.0
0b110 4.0
0b111 6.0
0b1000 -0.0
0b1001 -0.75
0b1010 -1.0
0b1011 -1.5
0b1100 -2.0
0b1101 -3.0
0b1110 -4.0
0b1111 -6.0


In [3]:
import math

def fp4_to_float_ieee_754(x: int) -> float:
    sign = (x >> 3) & 0b1
    exp_bits = (x >> 1) & 0b11
    frac = x & 0b1

    if exp_bits == 0 and frac == 0:
        return 0.0 * (-1) ** sign
    
    if exp_bits == 0b11:
        if frac == 0:
            return -math.inf if sign else math.inf
        else:
            return float("nan")

    E = exp_bits - BIAS
    mant = 1.0 + frac * 0.5
    return ((-1) ** sign) * mant * (2 ** E)

In [4]:
for x in range(1 << 4):
    print(bin(x), fp4_to_float_ieee_754(x))

0b0 0.0
0b1 0.75
0b10 1.0
0b11 1.5
0b100 2.0
0b101 3.0
0b110 inf
0b111 nan
0b1000 -0.0
0b1001 -0.75
0b1010 -1.0
0b1011 -1.5
0b1100 -2.0
0b1101 -3.0
0b1110 -inf
0b1111 nan


In [5]:


def decode_fp4(x: int):
    """FP4  (sign - 1, E - 2, mant_int - 1)
    """
    sign = (x >> 3) & 0b1
    exp_bits = (x >> 1) & 0b11
    frac = x & 0b1

    # Ноль: 0000
    if exp_bits == 0 and frac == 0:
        return sign, None, 0  #zero

    E = exp_bits - BIAS        
    mant_int = 2 + frac           
    return sign, E, mant_int


def encode_fp4(sign: int, E: int, mant_int: int) -> int:
   
    # zero
    if mant_int == 0:
        return 0

   
   
    
    while mant_int >= 4 and E < EMAX:
        mant_int >>= 1
        E += 1

    while mant_int > 0 and mant_int < 2 and E > EMIN:
        mant_int <<= 1
        E -= 1

    #infty
    if E > EMAX:
        exp_bits = 0b11
        frac = 0
        return (sign << 3) | (exp_bits << 1) | frac

    #zero-eps
    if E < EMIN:
        return 0

    exp_bits = E + BIAS
    frac = mant_int - 2  
    return (sign << 3) | (exp_bits << 1) | frac


def add_fp4(a: int, b: int) -> int:
    
    signA, EA, mantA = decode_fp4(a)
    signB, EB, mantB = decode_fp4(b)

    if mantA == 0:
        return b
    if mantB == 0:
        return a

    
    if EA > EB:
        d = EA - EB
        mantB >>= d 
        EB = EA
        E = EA
    elif EB > EA:
        d = EB - EA
        mantA >>= d
        EA = EB
        E = EB
    else:
        E = EA  

   
    valA = mantA * (-1) ** signA
    valB = mantB * (-1) ** signB
    res = valA + valB

    
    if res == 0:
        return 0

    sign_res = 0 if res > 0 else 1
    mant_res = abs(res)

    
    return encode_fp4(sign_res, E, mant_res)




In [7]:
for a in range(0, 1 << 4):
    for b in range(1 << 4):
        print(fp4_to_float(a), fp4_to_float(b),
               fp4_to_float(add_fp4(a, b)))
        print(bin(a), bin(b), bin(add_fp4(a, b)))
        print("----------")

0.0 0.0 0.0
0b0 0b0 0b0
----------
0.0 0.75 0.75
0b0 0b1 0b1
----------
0.0 1.0 1.0
0b0 0b10 0b10
----------
0.0 1.5 1.5
0b0 0b11 0b11
----------
0.0 2.0 2.0
0b0 0b100 0b100
----------
0.0 3.0 3.0
0b0 0b101 0b101
----------
0.0 4.0 4.0
0b0 0b110 0b110
----------
0.0 6.0 6.0
0b0 0b111 0b111
----------
0.0 -0.0 -0.0
0b0 0b1000 0b1000
----------
0.0 -0.75 -0.75
0b0 0b1001 0b1001
----------
0.0 -1.0 -1.0
0b0 0b1010 0b1010
----------
0.0 -1.5 -1.5
0b0 0b1011 0b1011
----------
0.0 -2.0 -2.0
0b0 0b1100 0b1100
----------
0.0 -3.0 -3.0
0b0 0b1101 0b1101
----------
0.0 -4.0 -4.0
0b0 0b1110 0b1110
----------
0.0 -6.0 -6.0
0b0 0b1111 0b1111
----------
0.75 0.0 0.75
0b1 0b0 0b1
----------
0.75 0.75 1.5
0b1 0b1 0b11
----------
0.75 1.0 1.5
0b1 0b10 0b11
----------
0.75 1.5 2.0
0b1 0b11 0b100
----------
0.75 2.0 2.0
0b1 0b100 0b100
----------
0.75 3.0 3.0
0b1 0b101 0b101
----------
0.75 4.0 4.0
0b1 0b110 0b110
----------
0.75 6.0 6.0
0b1 0b111 0b111
----------
0.75 -0.0 0.75
0b1 0b1000 0b1
----------